<a href="https://colab.research.google.com/github/satishgc227/git-test/blob/master/OTTO_RECOMMENDER_SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task overview :
The aim of this competition is to predict e-commerce clicks, cart additions, and orders. You'll build a multi-objective recommender system based on previous events in a user session.

Current recommender systems consist of various models with different approaches, ranging from simple matrix factorization to a transformer-type deep neural network. However, no single model exists that can simultaneously optimize multiple objectives. In this competition, you’ll build a single entry to predict click-through, add-to-cart, and conversion rates based on previous same-session events.

Import libraries:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys

In [ ]:
#!pip install zipfile36

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# if sys.version_info >= (3, 6):
#     import zipfile
# else:
#     import zipfile36 as zipfile


# file_name="otto-recommender-system.zip"

In [ ]:
# # opening the zip file in READ mode
# with zipfile(file_name, 'r') as zip:
#     # printing all the contents of the zip file
#     zip.printdir()
  
#     # extracting all the files
#     print('Extracting all the files now...')
#     zip.extractall()
#     print('Done!')

TypeError: ignored

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
df=pd.read_csv("/content/traian csv.csv")

In [3]:
df

,session,aid,ts,type
0,8000000,766953,1.660420e+12,clicks
1,8000000,1586093,1.660420e+12,clicks
2,8000000,1586093,1.660420e+12,clicks
3,8000000,682970,1.660420e+12,clicks
4,8000000,799794,1.660420e+12,clicks
...,...,...,...,...
89163,8008042,1305074,1.660420e+12,clicks
89164,8008042,1294458,1.660420e+12,clicks
89165,8008042,679541,1.660420e+12,clicks
89166,8008042,628622,1.660420e+12,clicks


Data preprocessing:

In [4]:
df.isnull().sum()

session    0
aid        0
ts         1
type       1
dtype: int64

In [5]:
df.dropna()

,session,aid,ts,type
0,8000000,766953,1.660420e+12,clicks
1,8000000,1586093,1.660420e+12,clicks
2,8000000,1586093,1.660420e+12,clicks
3,8000000,682970,1.660420e+12,clicks
4,8000000,799794,1.660420e+12,clicks
...,...,...,...,...
89162,8008042,1697998,1.660420e+12,clicks
89163,8008042,1305074,1.660420e+12,clicks
89164,8008042,1294458,1.660420e+12,clicks
89165,8008042,679541,1.660420e+12,clicks


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89168 entries, 0 to 89167
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   session  89168 non-null  int64  
 1   aid      89168 non-null  int64  
 2   ts       89167 non-null  float64
 3   type     89167 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 2.7+ MB


In [ ]:
# import json
# f=open("/content/test.jsonl")
# chunks=f.read()
# chunks

In [ ]:
#print(chunks)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# import pandas as pd
# import json
# import glob
# from pandas.io.json import json_normalize

# json_files = glob.glob("*.json")
# dfs = []
# for file in json_files:
#     with open("/content/test.jsonl") as f:
#         for line in f.readlines():
#             df = pd.json_normalize(json.loads(line))
#             list_= ['','Item.metadata.M.timestamp.S','Item.sensor.M.celcius.N','Item.sensor.M.water.N']
#             df = df.loc[:, df.columns.isin(list_)]
#             dfs.append(df)
# df_combine = pd.concat(dfs, sort=False)
# df_combine.to_csv('json_to_raw.csv',index= None)

ValueError: ignored

In [ ]:
# import zipfile
# from google.colab import drive

# drive.mount('/content/drive/')

# zip_ref = zipfile.ZipFile("/content/drive/Othercomputers/My Laptop/Downloads/train.jsonl.zip", 'r')
# zip_ref.extractall("/content/drive/Othercomputers/My Laptop/Downloads/tmp")
# zip_ref.close()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# import json
# dd=open("/content/drive/Othercomputers/My Laptop/Downloads/tmp/train.jsonl")

In [ ]:
# import pandas as pd
# chunks=pd.read_json(dd,lines=True, chunksize=1000)

In [ ]:
#fg=pd.DataFrame(chunks)

CPU times: user 9.57 ms, sys: 0 ns, total: 9.57 ms
Wall time: 9.43 ms


number of samples in train: 17,360
number of chunks: 1


CPU times: user 1.81 ms, sys: 0 ns, total: 1.81 ms
Wall time: 1.54 ms


In [19]:
sample = pd.DataFrame(df.loc[8000000]['events'])

KeyError: ignored